In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import random
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

from nltk.corpus import stopwords
import nltk

from sklearn.feature_extraction.text import CountVectorizer

e:\deep-learning-on-practice-NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yxtag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df_cv = pd.read_csv("../data/datasets_with_text/cv.csv")
df_vac = pd.read_csv("../data/datasets_with_text/vac.csv")

In [4]:
cvs = df_cv["cv_text"]
vacs = df_vac["vac_text"]

In [5]:
umap_model = UMAP(n_neighbors=5, n_components=192, metric='cosine', low_memory=False, random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('russian') + stopwords.words('english'), ngram_range=(1, 3))
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=5, metric='euclidean', prediction_data=True)

topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       nr_topics=400, top_n_words=10, language='multilingual', verbose=True)

topics, probs = topic_model.fit_transform(cvs[::5])

2023-12-21 03:05:23,451 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 280/280 [00:10<00:00, 25.52it/s]
2023-12-21 03:05:37,111 - BERTopic - Embedding - Completed ✓
2023-12-21 03:05:37,111 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-21 03:06:06,775 - BERTopic - Dimensionality - Completed ✓
2023-12-21 03:06:06,780 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-21 03:06:12,096 - BERTopic - Cluster - Completed ✓
2023-12-21 03:06:12,097 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-21 03:06:37,349 - BERTopic - Representation - Completed ✓
2023-12-21 03:06:37,378 - BERTopic - Topic reduction - Reducing number of topics
2023-12-21 03:06:37,380 - BERTopic - Topic reduction - Reduced number of topics from 312 to 312


In [6]:
vac_num = random.randint(0, len(df_vac) - 1)
vac = df_vac.iloc[vac_num]["vac_text"]

In [8]:
vac_topics, vac_probs = topic_model.transform([vac])

Batches: 100%|██████████| 1/1 [00:00<00:00, 12.35it/s]
2023-12-21 03:08:48,249 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2023-12-21 03:08:48,679 - BERTopic - Dimensionality - Completed ✓
2023-12-21 03:08:48,679 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2023-12-21 03:08:48,680 - BERTopic - Cluster - Completed ✓


In [10]:
vac_topics[0]

116

In [11]:
topic_model.get_topic(topic=vac_topics[0])

[('seo', 0.016616008743081084),
 ('яндекс', 0.007945751962789988),
 ('сайта', 0.007662596609921459),
 ('seo специалист', 0.007364561166807466),
 ('сайтов', 0.007218810347136281),
 ('оптимизация', 0.005531717782984154),
 ('google', 0.0054370687919524435),
 ('массаж', 0.005183419031173764),
 ('продвижение', 0.0049292186255787416),
 ('продвижение сайтов', 0.004082912057133549)]

In [14]:
print(df_vac.iloc[vac_num]["vac_text"])

Сфера деятельности:
Оптимизация сайта (SEO)
Должность:
SEO-оптимизатор / Интернет-маркетолог
Требования:
Понимание принципов юзабилити, пользовательского фактора. Понимание современных алгоритмов поисковых систем. Понимание принципов и опыт написания SEO и LSI текстов. 
Обязанности:
Внутренняя и внешняя SEO оптимизация сайтов. Проведение комплексного аудита проекта. Составление семантических ядер от 100 до +10000 запросов. 
Навыки:
SEO оптимизация,Составление семантического ядра,SEO
Описание вакансии:
Всем привет! Федеральная сеть интернет-магазинов в поисках своего Seo-оптимизатора. Мы занимаемся продажей разнообразной современной электроники, детских товаров, товаров для видеонаблюдения, охоты и рыбалки 100 городах РФ. Также в нашем активе есть собственная курьерская компания и множество новых, интересных проектов и планов. Мы уже успели реализовать множество интернет-магазинов и еще много чего планируем сделать. Мы ищем заинтересованного в развитии сотрудника, который будет стремить

In [16]:
cv_for_vac = topic_model.get_representative_docs()

In [18]:
cv_for_vac[vac_topics[0]]

['Пол, возраст:\nМужчина ,  29 лет , родился 24 ноября 1989\nЗП:\n250000 руб.\nИщет работу на должность::\nРуководитель SEO отдела\nГород, переезд, командировки:\nМосква ,  м. Новокосино , готов к переезду (Москва, Другие страны) , готов к командировкам\nЗанятость:\nполная занятость\nГрафик:\nгибкий график, полный день, удаленная работа\nОпыт работы:\nОпыт работы 11 лет 6 месяцев  Руководитель SEO отдела 250 000 руб. Информационные технологии, интернет, телеком Программирование, Разработка Оптимизация сайта (SEO) Электронная коммерция Занятость: полная занятость График работы: гибкий график, полный день, удаленная работа Опыт работы 11 лет 6 месяцев Февраль 2015 — по настоящее время 4 года 3 месяца Lamoda Москва , lamoda.ru Розничная торговля ... Розничная сеть (одежда, обувь, аксессуары) Товары народного потребления (непищевые) Одежда, обувь, аксессуары, текстиль (продвижение, оптовая торговля) Одежда, обувь, аксессуары, текстиль (производство) Head of SEO Автоматизация аналитики SEO 

In [20]:
topic_model.save("topic_model", serialization="safetensors", save_ctfidf=True)

In [21]:
topic_model.save("topic_model_pytorch", serialization="pytorch", save_ctfidf=True)